In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import logit

In [2]:
# Read data

In [3]:
df = pd.read_csv('data/full_game_rotation_data_of_playoffs_for_past_5_years.csv').drop(columns=['Unnamed: 0'])
df.IN_TIME_REAL = df.IN_TIME_REAL / 10.0
df.OUT_TIME_REAL = df.OUT_TIME_REAL / 10.0
df.MIN = df.MIN / 10.0

In [4]:
df.head()

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT,HOME_TEAM,SEASON_ID,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,WL,MIN,PTS
0,41900406,1610612747,L.A. Lakers,Lakers,2544,LeBron,James,0.0,488.0,6,4.0,0.211,0,42019,LAL,2020-10-11,LAL @ MIA,W,23.9,106
1,41900406,1610612747,L.A. Lakers,Lakers,2544,LeBron,James,583.0,1012.0,5,8.0,0.188,0,42019,LAL,2020-10-11,LAL @ MIA,W,23.9,106
2,41900406,1610612747,L.A. Lakers,Lakers,2544,LeBron,James,1155.0,1728.0,4,18.0,0.160,0,42019,LAL,2020-10-11,LAL @ MIA,W,23.9,106
3,41900406,1610612747,L.A. Lakers,Lakers,2544,LeBron,James,1810.0,2793.0,13,-12.0,0.333,0,42019,LAL,2020-10-11,LAL @ MIA,W,23.9,106
4,41900406,1610612747,L.A. Lakers,Lakers,2730,Dwight,Howard,2814.0,2880.0,3,-4.0,0.333,0,42019,LAL,2020-10-11,LAL @ MIA,W,23.9,106


# Create Features

In [5]:
# Find starting 5

In [6]:
df['STARTING_5'] = df.groupby(['GAME_ID', 'PERSON_ID'])['IN_TIME_REAL'].transform('min') == 0

In [7]:
# Find total playtime per game and player

In [8]:
df['DURATION'] = df.OUT_TIME_REAL - df.IN_TIME_REAL
df['PLAYTIME_PP_PG'] = df.groupby(['GAME_ID', 'PERSON_ID'])['DURATION'].transform('sum')

In [9]:
# Find avg player playtime per game per team

In [10]:
df['AVG_PLAYER_PLAYTIME_PG_PT'] = df[['GAME_ID', 'TEAM_ID', 'PERSON_ID', 'PLAYTIME_PP_PG']].drop_duplicates()\
                                    .groupby(['GAME_ID', 'TEAM_ID'])['PLAYTIME_PP_PG'].transform('mean')

In [11]:
# Find avg player playtime per game per team of starting 5

In [12]:
df['AVG_PLAYER_PLAYTIME_PG_PT_S5'] = df[df.STARTING_5][['GAME_ID', 'TEAM_ID', 'PERSON_ID', 'PLAYTIME_PP_PG']]\
                                    .drop_duplicates()\
                                    .groupby(['GAME_ID', 'TEAM_ID'])['PLAYTIME_PP_PG'].transform('mean')

In [13]:
# Find avg continuous player playtime per game per team

In [14]:
df['AVG_CNT_PLAYER_PLAYTIME_PG_PT'] = df.groupby(['GAME_ID', 'TEAM_ID'])['DURATION'].transform('mean')

In [15]:
# Find avg continuous player playtime per game per team of starting 5

In [16]:
df['AVG_CNT_PLAYER_PLAYTIME_PG_PT_S5'] = df[df.STARTING_5].groupby(['GAME_ID', 'TEAM_ID'])['DURATION']\
                                                       .transform('mean')

In [17]:
# For each playtime minute create a column and indicate if player was on the field at this moment

In [18]:
for m in range(48):
    if m+1 <10:
        df['MIN 0{}'.format(m+1)] = (df.IN_TIME_REAL/60.0 <= (m+1)) & (df.OUT_TIME_REAL/60.0 >= (m+1))
        df['MIN 0{}'.format(m+1)] = np.where(df['MIN 0{}'.format(m+1)], 1, 0)
    else:
        df['MIN {}'.format(m+1)] = (df.IN_TIME_REAL/60.0 <= (m+1)) & (df.OUT_TIME_REAL/60.0 >= (m+1))
        df['MIN {}'.format(m+1)] = np.where(df['MIN {}'.format(m+1)], 1, 0)

In [19]:
df['WIN'] = np.where(df.WL=='W', 1, 0)
df['BUBBLE'] = np.where(df.SEASON_ID.astype('str').str[1:] == '2019', 1, 0)

# Create DF for Viz

In [20]:
df_viz = df.groupby(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'SEASON_ID', 'MATCHUP', 'WIN', 
                     'BUBBLE', 'PERSON_ID', 'PLAYER_FIRST', 'PLAYER_LAST', 'TEAM_CITY', 'HOME_TEAM'])\
           .agg({'STARTING_5': 'mean',
                 'DURATION': 'mean',
                 'PLAYTIME_PP_PG': 'mean',
                 'MIN 01': 'max', 
                 'MIN 02': 'max', 
                 'MIN 03': 'max',
                 'MIN 04': 'max', 
                 'MIN 05': 'max', 
                 'MIN 06': 'max', 
                 'MIN 07': 'max', 
                 'MIN 08': 'max', 
                 'MIN 09': 'max', 
                 'MIN 10': 'max',
                 'MIN 11': 'max', 
                 'MIN 12': 'max', 
                 'MIN 13': 'max', 
                 'MIN 14': 'max', 
                 'MIN 15': 'max', 
                 'MIN 16': 'max', 
                 'MIN 17': 'max',
                 'MIN 18': 'max', 
                 'MIN 19': 'max', 
                 'MIN 20': 'max', 
                 'MIN 21': 'max', 
                 'MIN 22': 'max', 
                 'MIN 23': 'max', 
                 'MIN 24': 'max',
                 'MIN 25': 'max', 
                 'MIN 26': 'max', 
                 'MIN 27': 'max', 
                 'MIN 28': 'max', 
                 'MIN 29': 'max', 
                 'MIN 30': 'max', 
                 'MIN 31': 'max',
                 'MIN 32': 'max', 
                 'MIN 33': 'max', 
                 'MIN 34': 'max', 
                 'MIN 35': 'max', 
                 'MIN 36': 'max', 
                 'MIN 37': 'max', 
                 'MIN 38': 'max',
                 'MIN 39': 'max', 
                 'MIN 40': 'max', 
                 'MIN 41': 'max', 
                 'MIN 42': 'max', 
                 'MIN 43': 'max', 
                 'MIN 44': 'max', 
                 'MIN 45': 'max',
                 'MIN 46': 'max', 
                 'MIN 47': 'max',
                 'MIN 48': 'max'
                })\
           .reset_index()

In [21]:
df_viz.head()

,GAME_ID,TEAM_ID,TEAM_NAME,SEASON_ID,MATCHUP,WIN,BUBBLE,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,...,MIN 39,MIN 40,MIN 41,MIN 42,MIN 43,MIN 44,MIN 45,MIN 46,MIN 47,MIN 48
0,41500101,1610612739,Cavaliers,42015,CLE vs. DET,1,0,2210,Richard,Jefferson,...,1,1,1,1,0,0,0,0,0,0
1,41500101,1610612739,Cavaliers,42015,CLE vs. DET,1,0,2544,LeBron,James,...,1,1,1,1,1,1,1,1,1,1
2,41500101,1610612739,Cavaliers,42015,CLE vs. DET,1,0,2747,JR,Smith,...,0,0,0,0,1,1,1,1,1,1
3,41500101,1610612739,Cavaliers,42015,CLE vs. DET,1,0,201567,Kevin,Love,...,1,1,1,1,1,1,1,1,1,0
4,41500101,1610612739,Cavaliers,42015,CLE vs. DET,1,0,202389,Timofey,Mozgov,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_viz.to_csv('data/per_player_df_for_visualization.csv', index=False)

# Create DF for modeling

In [23]:
# DF per game & team
df_agg = df.groupby(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'SEASON_ID', 'MATCHUP', 'WL'])\
           .agg({'AVG_PLAYER_PLAYTIME_PG_PT': 'mean', 
                 'AVG_PLAYER_PLAYTIME_PG_PT_S5': 'mean',
                 'AVG_CNT_PLAYER_PLAYTIME_PG_PT': 'mean', 
                 'AVG_CNT_PLAYER_PLAYTIME_PG_PT_S5': 'mean',
                 'HOME_TEAM': 'mean',
                })\
           .reset_index()
df_agg['WIN'] = np.where(df_agg.WL=='W', 1, 0)
df_agg['BUBBLE'] = np.where(df_agg.SEASON_ID.astype('str').str[1:] == '2019', 1, 0)

# Filter Teams that participated in the Bubble
bubble_teams = df_agg[df_agg['BUBBLE']==1].TEAM_NAME.unique()
df_agg = df_agg[df_agg.TEAM_NAME.isin(bubble_teams)]

df_agg.to_csv('data/per_team&game_df_for_visualization.csv', index=False)

In [24]:
# Bubble vs pre-Bubble split

In [25]:
df_agg_bubble = df_agg[df_agg['BUBBLE']==1]
df_agg_pre_bubble = df_agg[df_agg['BUBBLE']==0]

In [26]:
df[df.TEAM_NAME=='Lakers'].groupby('SEASON_ID').nunique()

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,...,MIN 41,MIN 42,MIN 43,MIN 44,MIN 45,MIN 46,MIN 47,MIN 48,WIN,BUBBLE
SEASON_ID,,,,,,,,,,,,,,,,,,,,,
42019,21,1,1,1,15,15,15,349,349,22,...,2,2,2,2,2,2,2,2,2,1


# Descriptive Analysis

In [27]:
# Compare Bubble vs pre-Bubble
df_agg.groupby(['TEAM_NAME', 'BUBBLE'])\
           .agg({'AVG_PLAYER_PLAYTIME_PG_PT': 'mean', 
                 'AVG_PLAYER_PLAYTIME_PG_PT_S5': 'mean',
                 'AVG_CNT_PLAYER_PLAYTIME_PG_PT': 'mean', 
                 'AVG_CNT_PLAYER_PLAYTIME_PG_PT_S5': 'mean',
                 'HOME_TEAM': 'mean',
                })\
           .head(50)

AVG_PLAYER_PLAYTIME_PG_PT  AVG_PLAYER_PLAYTIME_PG_PT_S5  \
TEAM_NAME     BUBBLE                                                            
76ers         0                     1367.486756                   2021.442727   
              1                     1460.000000                   2079.875000   
Bucks         0                     1376.852314                   1878.193571   
              1                     1374.496503                   1845.454000   
Celtics       0                     1394.306719                   1905.366154   
              1                     1455.004799                   2203.898824   
Clippers      0                     1377.195436                   1813.722105   
              1                     1276.218397                   1847.210769   
Heat          0                     1402.749356                   1835.413684   
              1                     1554.495980                   1992.718095   
Jazz          0                     1374.690495                   1906.781481   
              1                     1511.721612                   2093.845714   
Lakers        1                     1352.807193                   1711.957143   
Magic         0                     1360.000000                   1832.288000   
              1                     1541.818182                   1835.272000   
Mavericks     0                     1381.818182                   1713.340000   
              1                     1403.181818                   2016.960000   
Nets          0                     1272.615385                   1567.176000   
              1                     1326.923077                   1945.515000   
Nuggets       0                     1537.489177                   2081.540000   
              1                     1439.938658                   1931.625263   
Pacers        0                     1413.070566                   1936.335455   
              1                     1487.272727                   2075.605000   
Raptors       0                     1406.490905                   1937.377500   
              1                     1506.081797                   2052.989091   
Rockets       0                     1469.947628                   2030.633182   
              1                     1612.781177                   2059.541667   
Thunder       0                     1397.350884                   2032.384706   
              1                     1533.598901                   1975.877143   
Trail Blazers 0                     1398.245088                   1937.896000   
              1                     1523.636364                   1945.968000   

                      AVG_CNT_PLAYER_PLAYTIME_PG_PT  \
TEAM_NAME     BUBBLE                                  
76ers         0                          390.397962   
              1                          466.711846   
Bucks         0                          405.684015   
              1                          269.251918   
Celtics       0                          429.021210   
              1                          468.887811   
Clippers      0                          418.818277   
              1                          389.236969   
Heat          0                          431.021021   
              1                          443.306225   
Jazz          0                          420.741444   
              1                          481.345993   
Lakers        1                          452.111785   
Magic         0                          471.036274   
              1                          480.213571   
Mavericks     0                          391.976024   
              1                          411.655127   
Nets          0                          417.249603   
              1                          546.080586   
Nuggets       0                          484.346645   
              1                          483.636899   
Pacers        0                          470.691390   
              1             

# Build Models

In [28]:
bubble_model = logit("""WIN ~ AVG_PLAYER_PLAYTIME_PG_PT
                            +AVG_PLAYER_PLAYTIME_PG_PT_S5
                            +AVG_CNT_PLAYER_PLAYTIME_PG_PT
                            +AVG_CNT_PLAYER_PLAYTIME_PG_PT_S5
                            +HOME_TEAM""", df_agg_bubble).fit()
print(bubble_model.summary())

Optimization terminated successfully.
         Current function value: 0.676448
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                    WIN   No. Observations:                  166
Model:                          Logit   Df Residuals:                      160
Method:                           MLE   Df Model:                            5
Date:                Sat, 14 Nov 2020   Pseudo R-squ.:                 0.02409
Time:                        14:45:50   Log-Likelihood:                -112.29
converged:                       True   LL-Null:                       -115.06
Covariance Type:            nonrobust   LLR p-value:                    0.3531
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            1.9788      1.383      1.43

In [29]:
pre_bubble_model = logit("""WIN ~ AVG_PLAYER_PLAYTIME_PG_PT
                            +AVG_PLAYER_PLAYTIME_PG_PT_S5
                            +AVG_CNT_PLAYER_PLAYTIME_PG_PT
                            +AVG_CNT_PLAYER_PLAYTIME_PG_PT_S5
                            +HOME_TEAM""", df_agg_pre_bubble).fit()
print(pre_bubble_model.summary())

Optimization terminated successfully.
         Current function value: 0.655333
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                    WIN   No. Observations:                  395
Model:                          Logit   Df Residuals:                      389
Method:                           MLE   Df Model:                            5
Date:                Sat, 14 Nov 2020   Pseudo R-squ.:                 0.04977
Time:                        14:45:50   Log-Likelihood:                -258.86
converged:                       True   LL-Null:                       -272.41
Covariance Type:            nonrobust   LLR p-value:                 5.421e-05
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -1.5911      1.029     -1.54